**Question: Are there differences in environmental exposure among patients with early-onset vs late-onset Asthma?**
* Find patients diagnosed with Asthma
* Find age of diagnosis for Asthma patients
* Find location for Asthma patients
* Find exposures by location for Asthma patients
* Compare exposures of patients diagnosed with Asthma during childhood vs during adulthood

**Data Sources**
* HUSH+ synthetic data resource
* [FHIR synthetic data resource](http://ictrweb.johnshopkins.edu/ictr/synthetic/)
* [UMLS](https://www.nlm.nih.gov/research/umls/), NOTE: there is a Web API [here](https://documentation.uts.nlm.nih.gov/rest/home.html) 
* [OHDSI Web API](http://www.ohdsi.org/web/wiki/doku.php?id=documentation:software:webapi)
* [Open AQ Platform API](https://docs.openaq.org), An API for open air quality data (pm25)
* [Green Translator Exposures API](https://exposures.renci.org/v1/ui), An API for pm25 and o3 measures

### Function and dataset definitions

In [2]:
## All the imports we need
import urllib2
from urllib2 import Request, urlopen, URLError
from urllib import quote_plus

import mysql.connector

import pprint, json, requests

from datetime import datetime, timedelta
from dateutil.parser import parse as parse_date
from greentranslator.api import GreenTranslator
query = GreenTranslator ().get_query ()

import dateutil

import zipcode

import string

%matplotlib inline
import numpy as np
import pandas as pd
import seaborn as sns

In [3]:
#connect to UMLS
try:
    cnx = mysql.connector.connect(user='tadmin',
                                password='ncats_translator!',
                                database='umls',
                                host='translator.ceyknq0yekb3.us-east-1.rds.amazonaws.com')
except mysql.connector.Error as err:
    if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
        print("Something is wrong with your user name or password")
    elif err.errno == errorcode.ER_BAD_DB_ERROR:
        print("Database does not exist")
    else:
        print(err)
else:
    print
    #cnx.close()

#### UMLS and OHDSI functions

In [4]:
## Given disease/condition term, get back ICD codes from OHDSI
def findICD_ohdsi(txt, icd_version = 9):
    if icd_version == 9:
        icd_type = 'ICD9CM'
    elif icd_version == 10:
        icd_type = 'ICD10'
    else: raise Exception("Invalid ICD version specified")    
    url_con = "http://api.ohdsi.org/WebAPI/vocabulary/search"
    headers = {'content-type': 'application/json'}
    params = {"QUERY": txt,
              "VOCABULARY_ID": [icd_type]}
    response = requests.post(url_con, data=json.dumps(params), headers=headers)
    data= json.loads(response.text.decode('utf-8'))
    return [d["CONCEPT_CODE"] for d in data]
print findICD_ohdsi('asthma')

# Get ICD10/ICD9 code for a given string from UMLS. By default we get back ICD10.
def findICD_umls(name, icd_version = 10):
    if icd_version == 9:
        icd_type = 'ICD9CM'
    elif icd_version == 10:
        icd_type = 'ICD10'
    else: raise Exception("Invalid ICD version specified")

    cursor = cnx.cursor()
    query = ("SELECT CUI FROM umls.MRCONSO WHERE STR='"+name+"'")
    cursor.execute(query, ())
    res = "Undef"
    for code in cursor:
        if res=="Undef":
            res = code
    if res != "Undef":
        query = ("SELECT CODE FROM umls.MRCONSO WHERE SAB='"+icd_type+"' AND CUI='"+res[0]+"'")
        cursor.execute(query, ())
        icd10 = "Undef"
        for code in cursor:
            icd10 = code
        return (icd10[0])
    return ("Undef")

[u'E945.7', u'493', u'493.9', u'493.90', u'493.92', u'493.91', u'493.2', u'493.20', u'493.22', u'493.21', u'493.82', u'493.0', u'493.00', u'493.02', u'493.01', u'V17.5', u'493.1', u'493.10', u'493.12', u'493.11', u'493.8', u'975.7']


#### Exposure data source founctions

In [5]:
#Function takes date of diagnosis, and location as input and returns degree of exposure 

def findExposureAtDateOfDx(dxdate,lon,lat):
    #dxnextdate = dateutil.parser.parse(dxdate) + timedelta(days=1)
    #dxnextdate = datetime.date(dxnextdate)
    pm25score = query.expo_get_scores (exposure_type = 'pm25',
                            start_date = dxdate,
                            end_date = dxdate,
                            exposure_point = ''+lon+','+lat+'')
    return(pm25score) 

def findAirQualityAtDateOfDx(dxdate,lat,lon):
    try:
        url = "https://api.openaq.org/v1/measurements?coordinates="+str(lat)+','+str(lon)+"&date="+str(dxdate)+"&parameter=pm25"
        #print url
        txt = urlopen(url).read()
        #print txt
    except Exception, e:
        raise Exception(e)
    exps= json.loads(txt)
    #pprint.pprint(exps)
    return(exps)

#### HUSH+ and FHIR synthetic data source functions

In [6]:
## Functions to retreive patients from different sources - HUSH+ & FHIR synthetic
def findPatients_fhir(code):
    try:
        url ="http://ictrweb.johnshopkins.edu/rest/synthetic/Condition?icd_10="+code+"&_count=999"
        txt = urlopen(url).read()
    except Exception, e:
        raise Exception(e)
    stuff = json.loads(txt)
    #print stuff
    return(stuff)

def findPatients_unc(age='8', sex='male', race='white', location='OUTPATIENT'):
    query = GreenTranslator ().get_query()
    return query.clinical_get_patients(age, sex, race, location)

In [7]:
#Function to retrieve age at diagnosis & location for patients from FHIR synthetic
def findDxAgeLocation_fhir(stuff):
    #pprint.pprint(stuff)
    res = []
    if stuff==[]:
        return []
    for p in stuff['entry']:
        #pprint.pprint(p)
        d1 = p['assertedDate']
        id = p['subject']['reference'][8:]
        try:
            url = 'http://ictrweb.johnshopkins.edu/rest/synthetic/Patient/'+id
            txt = urlopen(url).read()
        except:
            print "ictr api doesn't work"
        st = json.loads(txt)
        #pprint.pprint(st)
        d2 = st['birthDate']
        z = st['address'][0]['postalCode']
        datedx = d2 #e.g. 1925-12-08 00:00:00
        d1 = dateutil.parser.parse(d1)
        d2 = dateutil.parser.parse(d2)
        #print d1
        #print d2
        diff = d1 - d2
        #print diff
             
        obj = zipcode.isequal(z.encode("ascii", "ignore"))
        if obj is not None:
            d = obj.to_dict()
            lat = d['lat']
            lon = d['lon']
        else:
            lat = None
            lon = None
        t = (id,diff.days,datedx,lat,lon)         
        res.append(t)
    return(res)

#Add function to retrieve age at diagnosis & location for patients from HUSH+

### "What is the age range of Asthma diagnoses, grouped by location?"

In [12]:
asthmaCodes = findICD_umls("asthma") # We go with ICD10 codes

## Get age of diagnosis & locations for FHIR synthetic patients.
p_agelocs = []

print "Codes"
print len(asthmaCodes)
print asthmaCodes
for icd in asthmaCodes:
    print "ICD"
    print icd
    try:
        p = findPatients_fhir(asthmaCodes)
        print "Patients"
        pprint.pprint(p)
        # patients with icd exist
    except:
        print "no patients with icd"
        exit()
    p_ageloc = findDxAgeLocation_fhir(p)
    #pprint.pprint(p_ageloc)
    
    for dx in p_ageloc:
        num = int(dx[0])
        ageOfDxInDays = int(dx[1])
        dateAtDx = str(dx[2])
        lat = dx[3]
        lon = dx[4]
        coordinates=''+str(lat)+','+str(lon)+''
        #print coordinates
        p_agelocs.append((num, ageOfDxInDays,dateAtDx,lat,lon,coordinates))
        #p_agelocs.append(dx)
    #print p_agelocs[1]
    p_agelocs_df = pd.DataFrame(p_agelocs, columns=['id', 'ageOfDxInDays', 'dateAtDx', 'lat', 'lon','coordinates'])
    #print  p_agelocs_df
    
    s = p_agelocs_df['ageOfDxInDays']
    print s.describe()
    
    g = sns.factorplot(x="coordinates", y="ageOfDxInDays", data=p_agelocs_df, kind="box", size=3, aspect=2)
    g.set_xticklabels(rotation=50)


## Add code to get age of diagnosis & locations for HUSH+ patients.

Codes
5
J45.9
ICD
J
Patients
[]
count     0
unique    0
Name: ageOfDxInDays, dtype: int64


ValueError: min() arg is an empty sequence

### "What is the age range of Asthma diagnoses, grouped by air quality threshold?"
NOTE: will need to find literature-based thresholds for high & low air quality

In [26]:
##For date of diagnosis and location, what is air quality (pm25)?

#findExposureAtDateOfDx("2010-01-07","35.9131996","-79.0558445")
#findAirQualityAtDateOfDx("2010-01-07","35.91","-79.05")

pprint.pprint(p_agelocs)
pprint.pprint(p_agelocs_df)

for item, p in p_agelocs_df.iterrows():
    #print "item, p"
    #print item
    #print p
    dateDx = p['dateAtDx']
    lat = p['lat']
    lon = p['lon']
    #print dateDx
    #print lat
    txt =''
    try:
        txt = findAirQualityAtDateOfDx(dateDx,lat,lon)
        #findExposureAtDateOfDx(dateDx,lon,lat)
    except:
        print "location not found"
        #exit()
    if len(txt)> 0:
        try:
            p_exp = json.loads(txt)
        except:
            p_exp = []
    else:
        p_exp = []
    #pm25 = p_exp['value']
    #print pm25


[(178639511, 5223, '1940-10-26', 38.87, -77.1, '38.87,-77.1'),
 (381740996, 3976, '1963-04-18', 38.94, -77.19, '38.94,-77.19'),
 (770375887, 4602, '1992-04-24', 38.89, -77.03, '38.89,-77.03'),
 (567452750, 2338, '1995-02-27', 38.89, -77.03, '38.89,-77.03'),
 (666544429, 6731, '1989-03-05', 38.89, -77.03, '38.89,-77.03'),
 (545538309, 12206, '1942-07-08', 38.8, -76.94, '38.8,-76.94'),
 (772243798, 14870, '1954-07-06', 38.87, -76.9, '38.87,-76.9'),
 (228368759, 16237, '1958-05-31', 38.89, -77.03, '38.89,-77.03'),
 (383853998, 4840, '1942-09-26', 38.89, -77.03, '38.89,-77.03'),
 (383267724, 6341, '1991-01-29', 38.98, -77.08, '38.98,-77.08'),
 (561513328, 17677, '1952-10-05', 38.87, -77.1, '38.87,-77.1'),
 (933538820, 10463, '1985-08-17', 38.89, -77.03, '38.89,-77.03'),
 (773183620, 21689, '1949-01-21', 38.89, -77.03, '38.89,-77.03'),
 (248509890, 10033, '1952-11-08', 38.95, -76.95, '38.95,-76.95'),
 (809015918, 10913, '1976-06-24', 38.89, -77.03, '38.89,-77.03'),
 (858150638, 829, '1968-1

 (205420694, 12641, '1952-07-08', 38.98, -77.0, '38.98,-77.0'),
 (178093724, 4745, '1993-05-27', 38.89, -77.03, '38.89,-77.03'),
 (160122291, 11487, '1983-09-14', 38.89, -77.03, '38.89,-77.03'),
 (251317163, 8343, '1976-04-08', 38.89, -77.03, '38.89,-77.03'),
 (789992134, 5537, '1988-11-25', 38.89, -77.03, '38.89,-77.03'),
 (813729381, 2184, '1988-05-03', 38.87, -77.1, '38.87,-77.1'),
 (512201779, 8225, '1994-05-02', 38.95, -76.95, '38.95,-76.95'),
 (143108985, 12563, '1971-05-06', 38.89, -77.03, '38.89,-77.03'),
 (598286046, 878, '1960-11-29', 38.89, -77.03, '38.89,-77.03'),
 (589733080, 805, '1990-01-04', 38.89, -77.03, '38.89,-77.03'),
 (223541279, 11277, '1970-08-20', 38.95, -76.95, '38.95,-76.95'),
 (134117162, 5135, '1958-08-09', 38.89, -77.03, '38.89,-77.03'),
 (826725004, 8963, '1969-01-05', 38.95, -76.95, '38.95,-76.95'),
 (254600818, 4993, '1984-07-16', 38.89, -77.03, '38.89,-77.03'),
 (667831880, 12933, '1977-07-30', 38.89, -77.03, '38.89,-77.03'),
 (638120256, 19769, '1946-

 (725155754, 6698, '1980-11-09', 39.01, -77.02, '39.01,-77.02'),
 (934035160, 3403, '1988-07-17', 38.95, -76.95, '38.95,-76.95'),
 (729976805, 22565, '1941-12-26', 38.89, -77.03, '38.89,-77.03'),
 (395731954, 5768, '1957-01-19', 38.95, -76.95, '38.95,-76.95'),
 (447163697, 12335, '1968-08-13', 38.98, -77.08, '38.98,-77.08'),
 (759651142, 3591, '1974-12-09', 38.89, -77.03, '38.89,-77.03'),
 (684793178, 13541, '1964-05-27', 38.89, -77.03, '38.89,-77.03'),
 (769696891, 2281, '1963-07-09', 38.89, -77.03, '38.89,-77.03'),
 (638120256, 14242, '1946-07-27', 38.89, -77.03, '38.89,-77.03'),
 (978269677, 647, '1990-03-11', 38.89, -77.03, '38.89,-77.03'),
 (703177720, 9835, '1977-03-23', 38.95, -76.95, '38.95,-76.95'),
 (608974664, 6199, '1956-06-16', 38.95, -76.95, '38.95,-76.95'),
 (457744644, 4406, '1989-02-27', 38.87, -77.1, '38.87,-77.1'),
 (237811778, 11574, '1969-08-02', 38.8, -76.94, '38.8,-76.94'),
 (980797042, 4085, '2002-12-30', 38.89, -77.03, '38.89,-77.03'),
 (380012260, 7801, '1971-

item, p
1
id                  381740996
ageOfDxInDays            3976
dateAtDx           1963-04-18
lat                     38.94
lon                    -77.19
coordinates      38.94,-77.19
Name: 1, dtype: object
1963-04-18
38.94
item, p
2
id                  770375887
ageOfDxInDays            4602
dateAtDx           1992-04-24
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 2, dtype: object
1992-04-24
38.89
item, p
3
id                  567452750
ageOfDxInDays            2338
dateAtDx           1995-02-27
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 3, dtype: object
1995-02-27
38.89
item, p
4
id                  666544429
ageOfDxInDays            6731
dateAtDx           1989-03-05
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 4, dtype: object
1989-03-05
38.89
item, p
5
id                 545538309
ageOfDxInDays          12206
dateAtDx    

item, p
40
id                  437186962
ageOfDxInDays            5373
dateAtDx           1969-12-01
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 40, dtype: object
1969-12-01
38.89
item, p
41
id                  412676963
ageOfDxInDays            7904
dateAtDx           1977-05-25
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 41, dtype: object
1977-05-25
38.89
item, p
42
id                  385887860
ageOfDxInDays           16162
dateAtDx           1962-08-18
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 42, dtype: object
1962-08-18
38.89
item, p
43
id                  796962715
ageOfDxInDays           16837
dateAtDx           1951-07-12
lat                     38.98
lon                    -77.12
coordinates      38.98,-77.12
Name: 43, dtype: object
1951-07-12
38.98
item, p
44
id                  140185585
ageOfDxInDays            3458
d

...

item, p
819
id                  512278766
ageOfDxInDays            9882
dateAtDx           1984-07-15
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 819, dtype: object
1984-07-15
38.89
item, p
820
id                  916295683
ageOfDxInDays            4179
dateAtDx           1990-02-20
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 820, dtype: object
1990-02-20
38.89
item, p
821
id                  889460213
ageOfDxInDays            6194
dateAtDx           1988-03-13
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 821, dtype: object
1988-03-13
38.89
item, p
822
id                  221428312
ageOfDxInDays           23650
dateAtDx           1936-03-18
lat                     38.98
lon                    -77.08
coordinates      38.98,-77.08
Name: 822, dtype: object
1936-03-18
38.98
item, p
823
id                  309132473
ageOfDxInDays         

item, p
855
id                  614385639
ageOfDxInDays            8199
dateAtDx           1976-04-26
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 855, dtype: object
1976-04-26
38.89
item, p
856
id                  850826088
ageOfDxInDays            3079
dateAtDx           1959-06-06
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 856, dtype: object
1959-06-06
38.89
item, p
857
id                  214128265
ageOfDxInDays            4334
dateAtDx           1972-02-23
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 857, dtype: object
1972-02-23
38.89
item, p
858
id                  815064810
ageOfDxInDays           11386
dateAtDx           1974-09-26
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 858, dtype: object
1974-09-26
38.89
item, p
859
id                  223865772
ageOfDxInDays         

item, p
893
id                  241403300
ageOfDxInDays           11122
dateAtDx           1950-03-11
lat                     38.98
lon                    -77.08
coordinates      38.98,-77.08
Name: 893, dtype: object
1950-03-11
38.98
item, p
894
id                  923068137
ageOfDxInDays            2466
dateAtDx           2002-11-14
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 894, dtype: object
2002-11-14
38.89
item, p
895
id                  550750435
ageOfDxInDays           25116
dateAtDx           1936-03-09
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 895, dtype: object
1936-03-09
38.89
item, p
896
id                  759090292
ageOfDxInDays            6094
dateAtDx           1955-04-01
lat                     38.95
lon                    -76.95
coordinates      38.95,-76.95
Name: 896, dtype: object
1955-04-01
38.95
item, p
897
id                  472285760
ageOfDxInDays         

item, p
933
id                  936282911
ageOfDxInDays            8082
dateAtDx           1975-01-03
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 933, dtype: object
1975-01-03
38.89
item, p
934
id                  398880518
ageOfDxInDays            1078
dateAtDx           1994-05-23
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 934, dtype: object
1994-05-23
38.89
item, p
935
id                  569436924
ageOfDxInDays           13768
dateAtDx           1969-12-04
lat                     38.95
lon                    -76.95
coordinates      38.95,-76.95
Name: 935, dtype: object
1969-12-04
38.95
item, p
936
id                  810930934
ageOfDxInDays           18370
dateAtDx           1940-02-26
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 936, dtype: object
1940-02-26
38.89
item, p
937
id                 265601514
ageOfDxInDays          

item, p
971
id                  348090146
ageOfDxInDays            7569
dateAtDx           1986-10-16
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 971, dtype: object
1986-10-16
38.89
item, p
972
id                  337076890
ageOfDxInDays             709
dateAtDx           1967-02-09
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 972, dtype: object
1967-02-09
38.89
item, p
973
id                  405315479
ageOfDxInDays           19690
dateAtDx           1946-12-17
lat                     38.89
lon                    -77.03
coordinates      38.89,-77.03
Name: 973, dtype: object
1946-12-17
38.89
item, p
974
id                  936378868
ageOfDxInDays           12240
dateAtDx           1960-12-07
lat                     39.01
lon                    -77.02
coordinates      39.01,-77.02
Name: 974, dtype: object
1960-12-07
39.01
item, p
975
id                  534178009
ageOfDxInDays         